In [1]:
from Crypto.PublicKey import RSA as rsa
from Crypto.Hash import SHA256
import os
import gmpy2
import random

In [2]:
#Set basic variables.
#Generate_New_Signer_Key is only needed if the real signer doesnt have a key stored yet.
Generate_New_Signer_Key = 0
message = b'Test this message'
total_ring_size = 5
key_size = 1024

In [3]:
#Generate a new key for the real signer
#Note fix where you store they key.  lol it pushes to github like this
if Generate_New_Signer_Key == 1:
    signer_key = rsa.generate(key_size, os.urandom)
    f = open('../../keys/mykey.der','wb')
    f.write(signer_key.exportKey('DER'))
    f.close()

In [4]:
def ring_public_keys(signer_count, key_size):
    #here we generate public keys and n for all members in the ring besides the true signer
    #in a live environment these values would be known and would not require a function
    b = 0
    circle_pub_keys = []
    circle_n = []
    while b < (signer_count - 1):
        circle_key = rsa.generate(key_size, os.urandom)
        circle_pub_keys.append(circle_key.e)
        circle_n.append(circle_key.n)
        b += 1
    return circle_pub_keys, circle_n
    
    

In [5]:
def scramble_ring(circle_pub_keys, circle_n):
    #This is meant to randomly order the ring before signing
    k = len(circle_pub_keys)
    b = 0
    scrambled_order = []
    scrambled_pub_keys = []
    scrambled_n = []
    while b < k:
        scrambled_order.append(b)
        b += 1
    random.shuffle(scrambled_order)
    for i in scrambled_order:
        scrambled_pub_keys = circle_pub_keys[scrambled_order[i]]
        scrambled_n = circle_n[scrambled_order[i]]
    return scrambled_pub_keys, scrambled_n
        
        

In [6]:
def sign_by_ring(public_key_set, public_n_set, v):
    #This will sign each output.  This does not execute the trap door function.
    for i in public_key_set:
        for j in public_n_set:
            s = int.from_bytes(os.urandom(key_size // 8),'big')
            e = gmpy2.powmod(s,i,j)
            v = v ^ e
    return v
    

In [7]:
def trap_door(u,v,n,private_key):
    #Execute the trapdoor function that should return v = E_k(u) same as above prior to all the ring signers
    step_1 = v ^ u
    s = gmpy2.powmod(step_1,private_key,n)
    return s

In [8]:
#def rotate_ring(ring_keys,ring_n,ring_size):
    #here we need to pick a random starting ring member and rotate the ring.
    #after the rotation we can hand the data to the verifier.
#    b = randbetween(0,ring_size)

In [9]:
#Hash and encrypt the message to generate v
k = SHA256.new(message)
k = k.digest()
#Generate random value
u = int.from_bytes(os.urandom(key_size // 8),'big')
v = int.from_bytes(k,'big') ^ u

In [10]:
public_key_set, public_n_set = ring_public_keys(total_ring_size, key_size)

In [11]:
#Here we add the real signer to the ring
g = open('../../keys/mykey.der','rb')
key = rsa.importKey(g.read())
public_key_set.append(key.e)
public_n_set.append(key.n)
g.close()

In [12]:
scrambled_pub_key,scrambled_n = scramble_ring(public_key_set,public_n_set)

In [13]:
step1_ring_sig = sign_by_ring(public_key_set, public_n_set, v)

In [14]:
#Figure out how to add everything into this once you figure out the first encryption.
final_signature = trap_door(u,step1_ring_sig,key.n,key.d)

In [15]:
print(v)

88079493704947932336458533395641605486868838079728173465694690124723425190285485267697088928824787722836149887754004495244828569902956100813782586338963447625453031223594985696984586229045085833238654297966672956618194215591176168986182485883793801546515881660187888295269704233516743133577125930306554379637


In [16]:
print(u)

88079493704947932336458533395641605486868838079728173465694690124723425190285485267697088928824787722836149887754004495244828569902956100813782586338963447625453031223594985696984586229045085833238654297966672956618194215591176169076412563362817396437555696575410511224583368688689858726385027070894602262156


In [17]:
print(final_signature)

102604619923898215726226695149947339043553732883732401101406987900615368926840442873486381310388256269355672862439681867846171258650895604191628786971066527058408217712657844356905815643493886612033188069092008217313994100194438420177106609077077896052219933529200272774206354140533051908762136672094656865344


### Verify a signature

In [59]:
h = SHA.new(message)
verifier = sig.new(key)
if verifier.verify(h, signature):
    print("The signature is authentic.")
else:
    print("The signature is not authentic.")

The signature is authentic.
